In [ ]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("no object is so beautiful that under certain conditions it will not look ugly oscar wilde rt …".split(), ":red_heart:"),
    ("asahd really is a grown man in the body of a 1 year old".split(), ":face_with_tears_of_joy:"),
    ("yoongi tweet hello im min fell on butt what the min".split(), ":face_with_tears_of_joy:"),
    ("will and jada on how they discipline their children".split(), ":clapping_hands:")
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {":red_heart:": 0, ":face_with_tears_of_joy:": 1, ":clapping_hands:": 2}  # Assign each tag with a unique index

emojis = tag_to_ix.keys()

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'no': 0, 'object': 1, 'is': 2, 'so': 3, 'beautiful': 4, 'that': 5, 'under': 6, 'certain': 7, 'conditions': 8, 'it': 9, 'will': 10, 'not': 11, 'look': 12, 'ugly': 13, 'oscar': 14, 'wilde': 15, 'rt': 16, '…': 17, 'asahd': 18, 'really': 19, 'a': 20, 'grown': 21, 'man': 22, 'in': 23, 'the': 24, 'body': 25, 'of': 26, '1': 27, 'year': 28, 'old': 29, 'yoongi': 30, 'tweet': 31, 'hello': 32, 'im': 33, 'min': 34, 'fell': 35, 'on': 36, 'butt': 37, 'what': 38, 'and': 39, 'jada': 40, 'how': 41, 'they': 42, 'discipline': 43, 'their': 44, 'children': 45}


In [ ]:
emojis_tensor = prepare_sequence(emojis, tag_to_ix)
emoji_embeddings = nn.Embedding(len(emojis), EMBEDDING_DIM)

emoji_embeds = emoji_embeddings(emojis_tensor)

class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.emojis = prepare_sequence(emojis, tag_to_ix)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.2256, -1.1444, -0.9467],
        [-1.0909, -1.1137, -1.0915],
        [-1.0793, -1.0959, -1.1211],
        [-1.1501, -1.1245, -1.0256],
        [-1.0497, -1.1142, -1.1339],
        [-1.1073, -1.1159, -1.0731],
        [-1.1279, -1.0934, -1.0752],
        [-1.1062, -1.0657, -1.1249],
        [-1.0460, -1.0454, -1.2136],
        [-1.0064, -1.0914, -1.2083],
        [-1.0241, -1.0949, -1.1832],
        [-1.0855, -1.1260, -1.0849],
        [-0.9959, -1.2458, -1.0703],
        [-1.1488, -1.1974, -0.9649],
        [-1.1161, -1.1826, -1.0052],
        [-1.1146, -1.1546, -1.0307],
        [-1.0780, -1.2106, -1.0169],
        [-1.0023, -1.0824, -1.2236]])


In [ ]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:

        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence([tags]*len(sentence), tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training


In [ ]:
with torch.no_grad():
    inputs = prepare_sequence("will and jada fell butt min".split(" "), word_to_ix)
    tag_scores = model(inputs)
    
    scores = torch.argmax(tag_scores, dim=1)
    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(scores)

tensor([1, 1, 1, 1, 1, 1])
